
Step 1: Import packages & stuff needed to generate the data for inversion

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Perhaps this step can be skipped by saving directly to the workspace

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#package instalation

#Installing PROSAIL
!pip install prosail

#latin hypercube stuff
#lets try to do a LHS
!pip install lhsmdu

#this package as a number of functions to deal with hyperspectral data
#!pip install pysptools

     |████████████████████████████████| 153kB 2.5MB/s 


In [ ]:
!pip install hyperopt 

Packages for RandomForest and other auxiliary stuff

In [ ]:
#the beutiful R like data frame
import pandas as pd

#the famous numpy 
import numpy as np

#PROSPECT+SAIL Radiative transfer mode package
import prosail

#Sampling design package
import lhsmdu

#a few more stuff for random
import random as rdm
import math


# import the regressor for gaussian processes and also the pre-set group of kernels
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,DotProduct,Matern,ExpSineSquared,RationalQuadratic


from sklearn import metrics

Bunch of functions needed for generating data

In [ ]:
def custom_prosail(cab,cw,cm,lai):
  import prosail
  #default parameters
  n= 1.
  car=10.
  cbrown=0.01
  typelidf=1 #this is the default option
  lidfa = -1 #leaf angle distribution parameter a and b
  lidfb=-0.15
  hspot= 0.01 #hotspot parameters - got this from R package https://www.rdocumentation.org/packages/hsdar/versions/0.4.1/topics/PROSAIL
  #sun and viewing angle
  tts=30. #observation and solar position parameters
  tto=10. 
  psi=0.
  #for now i put them by hand but they should be an input of a custom function
  #tts=sol_zen #solar zenith angle
  #tto=inc_zen #sensor zenith angle
  #psi=raa
  rho_out = prosail.run_prosail(n,
                                 cab,
                                 car,
                                 cbrown,
                                 cw,
                                 cm,
                                 lai,
                                 lidfa,
                                 hspot,
                                 tts,tto,psi,
                                 typelidf, lidfb,
                                 prospect_version="D",
                                 factor='SDR', 
                                 rsoil=.5, psoil=.5)
  return(rho_out)

def Prosail2S2(path2csv,spectra_input):
  #importing pandas
  import pandas as pd
  import numpy
  import numpy as np
  #upload a S2_Response.csv from https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses

  s2_table = pd.read_csv(path2csv,sep=";",decimal=",") #check if this is proper, regarding the sep and dec
  #chekc which row you are actually extracting

  s2_table_sel = s2_table[s2_table['SR_WL'].between(400,2500)] #selects all values between 400 and 2500
  spectra_input_df = pd.DataFrame(data=spectra_input,columns=["rho"],index=s2_table_sel.index) #transforms the input array into a pandas df with the column name rho and row.index = to the original input table

  
  rho_s2 = s2_table_sel.multiply(spectra_input_df['rho'],axis="index") #calculates the numerator
  w_band_sum = s2_table_sel.sum(axis=0,skipna = True) #calculates the denominator

  output = (rho_s2.sum(axis=0)/w_band_sum).rename_axis("ID").values #runs the weighted mean and converts the output to a numpy array

  return output[1:] #removes the first value because it represents the wavelength column

#please LOAD THTE FILE NOW
filepath="/content/drive/My Drive/S2_Response.csv"
#filepath="/content/S2_Responses_S2B.csv"
#filepath="/content/drive/My Drive/S2_Response.csv"


def Gen_spectra_data(traits):
  k = 1
  #pd_train_traits=traits
  #print(range(len(traits)))
  for i in range(len(traits)):
    #n_t = pd_train_traits["n"][i]
    cab_t = traits["cab"][i]
    #car_t = pd_train_traits["car"][i]
    #cbrown_t = pd_train_traits["cbrown"][i]
    cw_t = traits["cw"][i]
    cm_t = traits["cm"][i]
    lai_t = traits["lai"][i]

    if k == 1:
      tr_rho_s = custom_prosail(cab_t,cw_t,cm_t,lai_t)
      tr_rho_s = Prosail2S2(filepath,tr_rho_s)
      #plt.plot ( x, tr_rho_s, ':', label="Training prosail")
      #plt.legend(loc='best')
      
    if k > 1:
      tr_rho_t = custom_prosail(cab_t,cw_t,cm_t,lai_t)
      tr_rho_t = Prosail2S2(filepath,tr_rho_t)
      tr_rho_s = np.vstack((tr_rho_s,tr_rho_t))
      #plt.plot ( x, tr_rho_t, ':')

    k = k+1


  rho_samples=tr_rho_s


  return rho_samples




#Function to store the outputs into a table
column_names=["Model",
              "NSamples",
              "OutOfBag",
              "KFold_tr_samples",
              "KFold_vl_samples",
              "Variable",
              "Fold_nr",
              "ExplVar",
              "Max_err",
              "Mean_abs_Err",
              "Mean_sqr_err",
              #"Mean_sqr_lg_err",
              "Median_abs_err",
              "r2",
              "MAPE",
              "ModelName"]
              #"Mean_poiss_dev",
              #"Mean_gamma_dev"]
              #"Mean_tweed_dev"]

#mape is not existant in the package so we have to create it:
#https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn
#from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

  #Here, the file is used for saving
  #creating a df to receive the data


def calc_metrics(MDL,Samples,oob_samples,kf_tr,kf_vl,Variable,Fold,Ref,Pred,Modelname):

  out_list = {"Model":MDL,
              "NSamples":Samples,
              "OutOfBag":oob_samples,
              "KFold_tr_samples":kf_tr,
              "KFold_vl_samples":kf_vl,
              "Variable":Variable,
              "Fold_nr":Fold,
              "ExplVar": metrics.explained_variance_score(Ref, Pred),
              "Max_err": metrics.max_error(Ref, Pred),
              "Mean_abs_Err": metrics.mean_absolute_error(Ref, Pred),
              "Mean_sqr_err": metrics.mean_squared_error(Ref, Pred),
              "Median_abs_err" : metrics.median_absolute_error(Ref, Pred),
              "r2": metrics.r2_score(Ref, Pred),
              "MAPE": mean_absolute_percentage_error(Ref, Pred),
              "ModelName":Modelname}


  return out_list

Hyperparameter tuning section

In [ ]:
# define an objective function

# These are a bunch of methods of te hyperopt package to generate the search space
from hyperopt import hp

# these are the minimizing function (fmin), Tree-parzen estimator method, an evaluation function, trial method and status indicators
from hyperopt import fmin, tpe, space_eval, Trials, STATUS_OK, STATUS_FAIL
from sklearn.model_selection import cross_val_score


Step 2: Generate the data that is going to be used for training

In [ ]:
#first the trait-space

n_traits=4 #I will test on 4 varying traits: cab, car, cw,cm,lai

#generating a LHS hypercube (it uses a 0 to 1 interval that can be used as a multiplier against the different traits)
np.random.seed(0)


#the values here are 1 less than the max so i can add a value later to it
#max_n=1 
max_cab=121. #add 1
#max_car=44. #add 1
#max_cbrown= 9.99 #add 0.01
max_cw=0.008 #add 0.001 
max_cm=0.008 #0.001
max_lai = 9.9 #add 0.1

min_cab = 10.
min_cw = 0.002
min_cm = 0.002
min_lai = .5

#set the number of samples used for the optimization
n_samples = 1500

In [ ]:
#this is for the training dataset
df_metrics = pd.DataFrame(columns=column_names)
#this is for the out of bag
df_metrics_valid = pd.DataFrame(columns=column_names)

#a path to store the models 
#path2folder = "/content/drive/My Drive/DSRP_Lunch_outputs/Models/"
path2folder = "/content/drive/My Drive/RTM_Inversion/HyperParameterTunning/RandF/"

#generating trait space and data
LHS_train = lhsmdu.createRandomStandardUniformMatrix(n_traits,n_samples)
pd_trait = pd.DataFrame.transpose(pd.DataFrame(LHS_train))
pd_trait.columns = ["cab","cw","cm","lai"]

pd_trait["cab"]=pd_trait["cab"]*max_cab+min_cab
pd_trait["cw"] =pd_trait["cw"] *max_cw+min_cw
pd_trait["cm"] =pd_trait["cm"] *max_cm+min_cw
pd_trait["lai"]=pd_trait["lai"]*max_lai+min_lai

#this is the same as above but on numpy format
np_trait = pd_trait.iloc[:,:].values

#now we first generate the date in hyperspectral, convolute it to S2 while iterating through the entire given trait space
np_spect = Gen_spectra_data(pd_trait)[:,[1,2,3,4,5,6,8,11,12]]

#now we remove 10% of the data for out-of-bag validation of the final model while the cross-validation happens within the hyperparemeter tunning
index = list(range(len(np_spect)))
index10 = rdm.sample(index,math.ceil(len(index)*.1))
index90 = [x for x in index if x not in index10]

#selecting the groups for training (tr) and validation (vl)
tr_trait_df = np_trait[index90,]
tr_spect_df = np_spect[index90,]

vl_trait_df = np_trait[index10,]
vl_spect_df = np_spect[index10,]

Definining the optimization function and the search space

---

In [ ]:

#objective function

#first we define the function
N_FOLDS = 10
MAX_EVALS = 200

#here we can just fetch the data so we keep everything in one place
train_labels = tr_trait_df
train_features = tr_spect_df 

from sklearn.model_selection import KFold # import KFold

def objective(params, n_folds = N_FOLDS):
    """Objective function for Random forest Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    gp = GaussianProcessRegressor(**params, random_state = 42) #normalizing and adding noise does not increase the likelihood of sucessfull minimization

    #scores = cross_val_score(gp, X=train_features, y=train_labels, cv=10, scoring='neg_mean_absolute_error') #thi is the simple Mean abs
    #the cross_val_score function in GP seems to work out wonky so lets try explicitely creating it

    kf = KFold(n_splits=N_FOLDS,shuffle=True,random_state=42)

    scores = np.empty((0,n_traits)) #an empty arrray to holdout all the results of the kfoldind
    for train_index,test_index in kf.split(train_labels):
      x_train_k,x_test_k = train_features[train_index],train_features[test_index]
      y_train_k,y_test_k = train_labels[train_index],train_labels[test_index]

      gp.fit(x_train_k,y_train_k)
      outval = gp.predict(x_test_k)

      #storing all the MAE scores into a single place - this can be changed to anoter function
      score = [metrics.mean_absolute_error(y_test_k[:,0], outval[:,0]),
               metrics.mean_absolute_error(y_test_k[:,1], outval[:,1]),
               metrics.mean_absolute_error(y_test_k[:,2], outval[:,2]),
               metrics.mean_absolute_error(y_test_k[:,3], outval[:,3])]
      scores=np.append(scores,score) #just add the values to the end of the numpy and continues


    # Extract the best score
    best_score = max(abs(scores)) #using the abs here forces the function to minimize independently of the sign
    # if the the result is miniming

    # Loss must be minimized
    #loss = 1 - best_score 
    loss = best_score 

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

#distribution functions for the parameter sampling are here:
#https://github.com/hyperopt/hyperopt/wiki/FMin


#first we should create the kernels
#K_white = kernels.WhiteKernel()

#seems trivial to limit the maximum number of iterations within this base function
#https://scikit-optimize.github.io/stable/_modules/sklearn/gaussian_process/_gpr.html
#or alternitively call an altered version of the scipy.optimizer (or with an improved optimization technique)


#RBF,DotProduct,Matern,ExpSineSquared,RationalQuadratic
#K_rbf = 1.0*kernels.RBF()
K_rbf = RBF(length_scale_bounds="fixed")
#K_ratQ = kernels.RationalQuadratic() 
K_ratQ = RationalQuadratic(length_scale_bounds="fixed",alpha_bounds="fixed")
#k_mate = 1.0*kernels.Matern()
k_mate = Matern(length_scale_bounds="fixed")
#K_expsine = kernels.ExpSineSquared()
#K_expsine = ExpSineSquared(length_scale_bounds="fixed",periodicity_bounds="fixed")
K_dotprod = DotProduct()
K_dotprod = DotProduct(sigma_0_bounds="fixed")

#kernel_list = [K_ratQ,K_rbf,k_mate,K_expsine,K_dotprod]
#kernel_list = [RBF(1.0, length_scale_bounds="fixed")] #ahh! eureka, this is the version of RBF it utilized by default -> it only has to minimize one thing, basically its very slow because of that..
kernel_list = [K_rbf,K_ratQ,k_mate,K_dotprod]

space = {
    'n_restarts_optimizer': hp.choice('n_restarts_optimizer', range(10, 101, 10)), #to avoid values of 0
    'kernel': hp.choice('kernel', kernel_list),
    'normalize_y':hp.choice('normalize_y',[True,False])
}

for i in range(10, 101, 10):
  print(i, end=', ')

10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 

This now is the order for running

In [ ]:
#this loop is the actual testing
bayes_trials = Trials()

best_pgr = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials,verbose=1)

100%|██████████| 200/200 [05:39<00:00,  1.70s/it, best loss: 0.028436356979695123]


Checking the outputs

In [ ]:
best_pgr

{'kernel': 1, 'n_restarts_optimizer': 7, 'normalize_y': 0}